In [1]:
import pickle
import joblib
from langdetect import detect
import googletrans
from googletrans import Translator
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
#Import necessary tools
translator = Translator()
tf1 = pickle.load( open( "banksatfidf_vectorizer2.pkl", "rb" ) )
model=joblib.load("Bankmodel_Sentiment_Analysis_classification.sav")
ps =WordNetLemmatizer()

In [2]:
#Instance of vectorizer
tf1_new = TfidfVectorizer(analyzer='word', ngram_range=(2,2), stop_words = "english", lowercase = True,
                          max_features = 300, vocabulary = tf1.vocabulary_)
tf1_new.fit_transform(tf1.vocabulary_)

<350x350 sparse matrix of type '<class 'numpy.float64'>'
	with 273 stored elements in Compressed Sparse Row format>

In [3]:
#Define stop words
stop_words= ['yourselves', 'between', 'whom', 'itself', 'is', "she's", 'up', 'herself', 'here', 'your', 'each', 
             'we', 'he', 'my', "you've", 'having', 'in', 'both', 'for', 'themselves', 'are', 'them', 'other',
             'and', 'an', 'during', 'their', 'can', 'yourself', 'she', 'until', 'so', 'these', 'ours', 'above', 
             'what', 'while', 'have', 're', 'more', 'only', "needn't", 'when', 'just', 'that', 'were',  
             'very', 'should', 'any', 'y', 'isn', 'who',  'a', 'they', 'to', 'too', "should've", 'has', 'before',
             'into', 'yours', "it's", 'do', 'against', 'on',  'now', 'her', 've', 'd', 'by', 'am', 'from', 
             'about', 'further', "that'll", "you'd", 'you', 'as', 'how', 'been', 'the', 'or', 'doing', 'such',
             'his', 'himself', 'ourselves',  'was', 'through', 'out', 'below', 'own', 'myself', 'theirs', 
             'me', 'why', 'once',  'him', 'than', 'be', 'most', "you'll", 'same', 'some', 'with', 'few', 'it',
             'at', 'after', 'its', 'which', 'there','our', 'this', 'hers', 'being', 'did', 'of', 'had', 'under',
             'over','again', 'where', 'those', 'then', "you're", 'i', 'because', 'does', 'all']

In [4]:
#Detec tlanguage fucntion
def detect_language(text):
    lang = detect(text)
    if lang == 'ar':
        return 'arabic'
    elif lang == 'en':
        return 'english'
    elif lang == 'fr':
        return 'french'
    else:
        return 'Unknown'

#Translate text function
def translate_text(text,language):
    translated=""
    if language == 'arabic':
        translated=translator.translate(str(text), src=str('arabic'), dest=str('english')).text
    elif language == 'french':
        translated=translator.translate(str(text), src=str('french'), dest=str('english')).text
    elif language == 'english':
        translated=text
    else :
        translated='verify your text'
    return translated

In [5]:
#Function to preprocess text and predect sentiment
def predict_sentiment(texttopredict):
    tokens_without_sw = str(" ".join(word for word in texttopredict.split() if word not in (stop_words)))
    lemmatized_output = ' '.join([ps.lemmatize(w) for w in tokens_without_sw.split()])
    X1= tf1_new.transform([lemmatized_output]) 
    y_pred = model.predict(X1)
    return y_pred

In [17]:
def label_sentiment(prediction):
    return "positive" if prediction == 1 else "negative"

In [16]:
texts = [
    "كانوا غير محترمين مع الزبائن",
    "Excellent treatment and the most wonderful bank, thanks!",
    "Le service était horrible et lent",
    "The staff wasn't very helpful"
]

for text in texts:
    language = detect_language(text)
    translated = translate_text(text, language)
    prediction = predict_sentiment(translated)
    sentiment_label = label_sentiment(prediction[0])
    
    print(f"Texte original : {text}")
    if language != 'english':
        print(f"Traduction en anglais : {translated}")
    print(f"Sentiment : {sentiment_label}")
    print("-----")

Texte original : كانوا غير محترمين مع الزبائن
Traduction en anglais : They were not respected with customers
Sentiment : negative
-----
Texte original : Excellent treatment and the most wonderful bank, thanks!
Sentiment : positive
-----
Texte original : Le service était horrible et lent
Traduction en anglais : The service was horrible and slow
Sentiment : negative
-----
Texte original : The staff wasn't very helpful
Sentiment : negative
-----
